In [13]:
import json
import torch
from gensim.models import KeyedVectors
from gensim import matutils
import numpy as np

In [14]:
print('Loading Vocabulary and Vectors')
word2vec = KeyedVectors.load_word2vec_format(
    'data/word2vec/GoogleNews-vectors-negative300.bin', binary=True)
vocabulary = json.load(open('data/processed_data/processed_data.json', 'r'))

Loading Vocabulary and Vectors


In [51]:
scores = rouge.get_scores(
    generated_summary, reference_summary
)
print(scores)

tensor(1.6194, grad_fn=<MseLossBackward0>)

In [102]:
def word2vec_emb(sequence, word2vec, vocabulary):
    
    seq_tokens = []
    
    for key in sequence:
        try:
            seq_tokens.append(vocabulary['ind2word'][str(key.item())])
        except:
            seq_tokens.append(vocabulary['ind2word'][list(
                vocabulary['ind2word'].keys())[-1]])
    
    vectorized = []
    for word in seq_tokens:
        try:
            vectorized.append(word2vec[word])
        except:
            vectorized.append(np.zeros(300,))
    
    return np.array(vectorized)

In [103]:
def similarity_cosine(vec1, vec2):
    cosine_similarity = np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))
    print(cosine_similarity)
    print(spatial.distance.cosine(vec1, vec2))
    return cosine_similarity

In [104]:
def reward(target, generated, word2vec, vocabulary):
    '''
    Calculate the reward for the generated text
    '''
    target = word2vec_emb(target, word2vec, vocabulary)
    generated = word2vec_emb(generated, word2vec, vocabulary)
    
    reward = similarity_cosine(target.mean(axis=0), generated.mean(axis=0))

    return reward

In [123]:
answer = torch.tensor([[6100, 4948, 6066, 1336, 4264, 4093,   37, 3343, 2467, 5527, 3396, 4326,
          219, 5560, 5845, 1464, 3149, 2805, 4154, 1899, 5642]])

question = torch.tensor([[6100, 2440, 5128, 5570,  511, 2483, 4828, 3564,  282, 3869, 2614,    7,
         4783, 4595, 5613, 1385, 6032, 4369,  286, 2259, 1900]])

summary = torch.tensor([[6100, 5955, 2642, 6021, 1982, 3081, 3554, 3802, 4577, 5298, 5555, 1110,
         4515, 2661, 2299, 4606, 4956, 4007, 3281, 2625, 2206, 4335, 2730, 2925,
         4523, 1231, 4269,  427,  861, 4104, 2630, 2499, 5430, 4043, 5146, 4595,
         3772, 5545, 4852, 3325, 4324]])

gtSummary = torch.tensor([[405,  333,  880,  188,  624,  194,  996,  293,  244, 6099, 3275,   99,
           25,  244,  754,  636,  936, 1461, 3653,   51,   55,   56,   57,    1,
           58,    1,   59,    1,   60,   52,  244, 1367, 6099, 1368, 3654, 2424,
           76,   51, 3655,   52]])

In [159]:
from scipy import spatial

In [128]:
# torch.cat((answer[0], question[0]))
print(reward(target=torch.cat((answer[0], question[0])), generated=summary[0], word2vec=word2vec, vocabulary=vocabulary))
print(reward(target=gtSummary[0], generated=summary[0], word2vec=word2vec, vocabulary=vocabulary))

0.7565139467498903
0.6523351061715779


In [1]:
from rouge import Rouge
def rouge_scores(target, generated, word2vec, vocabulary):
    rouge = Rouge()
    tar = []
    ref = []

    for key in target:
        try:
            if vocabulary['ind2word'][str(key.item())] !='UNK':
                tar.append(vocabulary['ind2word'][str(key.item())])
        except:
            pass

    for key in generated:
        try:
            if vocabulary['ind2word'][str(key.item())] !='UNK':
                ref.append(vocabulary['ind2word'][str(key.item())])
        except:
            pass

    return rouge.get_scores(' '.join(ref), ' '.join(tar), avg=True)

In [158]:
rouge_scores(torch.cat((answer[0], question[0])), summary[0], word2vec, vocabulary)

40 visible one-half calling retinal open-angle missing elbows street hopeless absolutely S loss Depakene TEE acid minimal double recurrences opposite Cutting tends efforts settings hepatitis reflexes heart-lung Name 1 Oxygen endoscopy if Autonomic beat companies cervical Nonne-Milroy separates Fragile osteochondrodysplasias abnormally
- - - - - - 
40 running electrical Recurrent barley amyloid Rapid senses containers Evidence Hospitalization testing Joseph FODMAP x-rays fasting coated Southeast psoralen Sharing natural G pacing mouthwashes Rasagiline While psychiatric resistance new lights relieving babies Involved specially firm beat relationships instability cookies colloidal mechanisms


{'rouge-1': {'r': 0.025, 'p': 0.025, 'f': 0.024999995000001003},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.025, 'p': 0.025, 'f': 0.024999995000001003}}

In [138]:
Rouge().get_scores('I go to the beach','I go to the beach every data', avg=True)

{'rouge-1': {'r': 0.7142857142857143, 'p': 1.0, 'f': 0.8333333284722222},
 'rouge-2': {'r': 0.6666666666666666, 'p': 1.0, 'f': 0.7999999952000001},
 'rouge-l': {'r': 0.7142857142857143, 'p': 1.0, 'f': 0.8333333284722222}}

In [1]:
import os
import json
import random
import numpy as np
from six.moves import range

from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision

import lc_options
from utils import lc_utilities as utils

In [2]:
params = {
    'inputJson': "data/processed_data/processed_data.json",
    'useGPU': False,
    
    # A-Bot checkpoint
    'startFrom': "./checkpoints/exp_4/abot_ep_59.vd",
    
    # Q-Bot checkpoint should given if interactive dialog is required
    'qstartFrom': "./checkpoints/exp_4/qbot_ep_59.vd",
    
    'beamSize': 5,
}

# RNG seed
manualSeed = 1597
random.seed(manualSeed)
torch.manual_seed(manualSeed)
if params['useGPU']:
    torch.cuda.manual_seed_all(manualSeed)

print('Loading json file: ' + params['inputJson'])
with open(params['inputJson'], 'r') as fileId:
    info = json.load(fileId)

wordCount = len(info['word2ind'])
# Add <START> and <END> to vocabulary
info['word2ind']['<START>'] = wordCount + 1
info['word2ind']['<END>'] = wordCount + 2
startToken = info['word2ind']['<START>']
endToken = info['word2ind']['<END>']
# Padding token is at index 0
vocabSize = wordCount + 3
print('Vocab size with <START>, <END>: %d' % vocabSize)

# Construct the reverse map
info['ind2word'] = {
    int(ind): word
    for word, ind in info['word2ind'].items()
}


Loading json file: data/processed_data/processed_data.json
Vocab size with <START>, <END>: 6055


In [3]:
def loadModel(params, agent='abot'):
    # should be everything used in encoderParam, decoderParam below
    encoderOptions = [
        'encoder', 'vocabSize', 'embedSize', 'rnnHiddenSize', 'numLayers',
        'useHistory', 'numRounds', 'dropout'
    ]
    decoderOptions = [
        'decoder', 'vocabSize', 'embedSize', 'rnnHiddenSize', 'numLayers',
        'dropout'
    ]
    modelOptions = encoderOptions + decoderOptions

    mdict = None
    gpuFlag = params['useGPU']
    startArg = 'startFrom' if agent == 'abot' else 'qstartFrom'
    assert params[startArg], "Need checkpoint for {}".format(agent)

    if params[startArg]:
        print('Loading model (weights and config) from {}'.format(
            params[startArg]))

        if gpuFlag:
            mdict = torch.load(params[startArg])
        else:
            mdict = torch.load(params[startArg],
                map_location=lambda storage, location: storage)

        # Model options is a union of standard model options defined
        # above and parameters loaded from checkpoint
        modelOptions = list(set(modelOptions).union(set(mdict['params'])))
        for opt in modelOptions:
            if opt not in params:
                params[opt] = mdict['params'][opt]

            elif params[opt] != mdict['params'][opt]:
                # Parameters are not overwritten from checkpoint
                pass

    # Initialize model class
    encoderParam = {k: params[k] for k in encoderOptions}
    decoderParam = {k: params[k] for k in decoderOptions}

    encoderParam['startToken'] = encoderParam['vocabSize'] - 2
    encoderParam['endToken'] = encoderParam['vocabSize'] - 1
    decoderParam['startToken'] = decoderParam['vocabSize'] - 2
    decoderParam['endToken'] = decoderParam['vocabSize'] - 1

    if agent == 'abot':
        encoderParam['type'] = params['encoder']
        decoderParam['type'] = params['decoder']
        encoderParam['isAnswerer'] = True
        from lc.models.lc_answerer import Answerer
        model = Answerer(encoderParam, decoderParam)

    elif agent == 'qbot':
        encoderParam['type'] = params['qencoder']
        decoderParam['type'] = params['qdecoder']
        encoderParam['isAnswerer'] = False
        from lc.models.lc_questioner import Questioner
        model = Questioner(
            encoderParam,
            decoderParam,
            summFeatureSize=40)

    # if params['useGPU']:
    #     model.cuda()

    if mdict:
        model.load_state_dict(mdict['model'])
        
    print("Loaded agent {}".format(agent))
    return model

In [4]:
torch.cuda.is_available()

False

In [5]:
aBot = None
qBot = None



In [6]:
# load aBot
if params['startFrom']:
    aBot = loadModel(params, 'abot')
    assert aBot.encoder.vocabSize == vocabSize, "Vocab size mismatch!"
    aBot.eval()
    
# load qBot
if params['qstartFrom']:
    qBot = loadModel(params, 'qbot')
    assert qBot.encoder.vocabSize == vocabSize, "Vocab size mismatch!"
    qBot.eval()

Loading model (weights and config) from ./checkpoints/exp_4/abot_ep_59.vd
Encoder: hre-ques-lateim-hist
Decoder: gen
Loaded agent abot
Loading model (weights and config) from ./checkpoints/exp_4/qbot_ep_59.vd
Encoder: hre-ques-lateim-hist
Decoder: gen
Loaded agent qbot


In [7]:
doc = 'Smoking and surgery Surgery - quitting smoking Surgery - quitting tobacco Wound healing - smoking Summary Quitting smoking and other tobacco products before surgery can improve your recovery and outcome after surgery. Most people who successfully quit smoking have tried and failed many times. DO NOT give up. Learning from your past tries can help you succeed. There Are Many Reasons to Quit Smoking Tar, nicotine, and other chemicals from smoking can increase your risk of many health problems. These include heart and blood vessel problems, such as: Blood clots and aneurysms in the brain, which can lead to strokes Coronary artery disease, including chest pain (angina) and heart attacks High blood pressure Poor blood supply to the legs Problems with erections Smoking also increases your risk for different types of cancer, including cancer of the: Lungs Mouth Larynx Esophagus Bladder Kidneys Pancreas Cervix Smoking also leads to lung problems, such as emphysema and chronic bronchitis, and makes asthma harder to control. Some smokers switch to smokeless tobacco instead of quitting tobacco completely. But using smokeless tobacco still carries health risks, such as: Developing mouth or nasal cancer Gum problems, tooth wear, and cavities Worsening high blood pressure and chest pain How Smoking Affects Surgery Smokers who have surgery have a higher chance than nonsmokers of blood clots forming in their legs. These clots may travel to and damage the lungs. Smoking decreases the amount of oxygen that reaches the cells in your surgical wound. As a result, your wound may heal more slowly and is more likely to become infected. All smokers carry an increased risk for heart and lung problems. Even when your surgery goes smoothly, smoking causes your body, heart, and lungs to work harder than if you did not smoke. Making the Decision to Quit Most doctors will tell you to stop using cigarettes and tobacco at least 4 weeks before your surgery. Stretching the time between quitting smoking and your surgery out to at least 10 weeks can decrease your risk of problems even more. Like any addiction, quitting tobacco is difficult. There are many ways to quit smoking and many resources to help you, such as: Family members, friends, and coworkers may be supportive or encouraging. Talk to your doctor about medicines, such as nicotine replacement and prescription medicines. If you join smoking cessation programs, you have a much better chance of success. Such programs are offered by hospitals, health departments, community centers, and work sites. Using nicotine gum around the time of surgery is not encouraged. The nicotine will still interfere with the healing of your surgical wound and have the same effect on your general health as using cigarettes and tobacco. Review Date 9/17/2016 Updated by: Debra G. Wechter, MD, FACS, general surgery practice specializing in breast cancer, Virginia Mason Medical Center, Seattle, WA. Also reviewed by David Zieve, MD, MHA, Isla Ogilvie, PhD, and the A.D.A.M. Editorial team. '
docs = doc
summary = 'Nicotine use can have many different effects on the body. It can: - Decrease the appetite; fear of weight gain makes some people unwilling to stop smoking - Boost mood, give people a sense of well-being, and possibly even relieve minor depression - Increase activity in the intestines - Create more saliva and phlegm - Increase the heart rate by around 10 to 20 beats per minute - Increase blood pressure by 5 to 10 mm Hg - Possibly cause sweating, nausea, and diarrhea - Stimulate memory and alertness; people who use tobacco often depend on it to help them accomplish certain tasks and perform well Symptoms of nicotine withdrawal appear within 2 to 3 hours after you last use tobacco.'

In [8]:
ind_map = lambda words: np.array([info['word2ind'].get(word, info['word2ind']['UNK']) 
                                  for word in words], dtype='int64')

tokenize = lambda string: ['<START>'] + word_tokenize(string) + ['<END>']

In [9]:
# Process document
doc_tokens = tokenize(doc)
doc = ind_map(doc_tokens)

In [26]:
doc

array([6053,  883,   32,  325, 1193,  409,  902,  871, 1193,  409,  902,
        834, 6052, 1954,  409,  871,  654, 2959,  871,   32,  402,  834,
        306,  515,  325,  203, 1024,  235,  132,   32, 5264,  315,  325,
         42,  301,   34,  397, 3457, 1029,  871,   21, 1052,   32, 3982,
         23,  556,   42, 2490, 2491, 1112, 1037,   42, 1555,   61,  235,
       2595, 6052,  203,  138,  244, 5265,   42,  600, 4154,  346, 6052,
          5, 1028,  883, 6052,    1,  830,    1,   32,  402,  355,   61,
        871,  203,  522,  235,  523,   39,   23,  391,  142,   42,  833,
        621,  524,   32,   99,  872,  142,    1,  249,  250,  194,  873,
        874,   32, 6052,   80,   62,  683,    1,  145,  203,  571,    5,
        875,  876,  877,  278,    1,  649,  575,  328,   50,  878,   52,
         32,  524,  460,  521,   99,  754,  879,   99,  880,    5,   62,
        881,  882,   14, 6052,  883,  124,  675,  235,  523,   12,  884,
        885,   39,  886,    1,  649,  886,   39,   

In [10]:
# Helper functions for converting tensors to words
to_str_pred = lambda w, l: str(" ".join([info['ind2word'][x] for x in list( filter(
        lambda x:x>0,w.data.cpu().numpy()))][:l.data.cpu()[0]]))[8:]
to_str_gt = lambda w: str(" ".join([info['ind2word'][x] for x in filter(
        lambda x:x>0,w.data.cpu().numpy())]))[8:-6]

In [11]:
def var_map(tensor):
    return Variable(tensor.unsqueeze(0), volatile=True)

In [12]:
doc_tensor = var_map(torch.from_numpy(doc))
doc_lens = var_map(torch.LongTensor([len(doc)]))

C:\Users\anest\AppData\Local\Temp\ipykernel_16912\881476409.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(tensor.unsqueeze(0), volatile=True)


In [41]:
if aBot:
    aBot.eval(), aBot.reset()
    aBot.observe(
        -1, document=doc_tensor, documentLens=doc_lens)
    
if qBot:
    qBot.train(), qBot.reset()
    qBot.observe(-1, document=doc_tensor, documentLens=doc_lens)

numRounds = 10
beamSize = 5
summ, summLens = qBot.predictSummary(
            beamSize=beamSize, inference='greedy')
print('----------------------------------------')
print("Summary: ", to_str_gt(summ[0]))    
for round in range(numRounds):
    questions, quesLens = qBot.forwardDecode(
            beamSize=beamSize, inference='greedy')
    qBot.observe(round, ques=doc_tensor, quesLens=doc_lens)
    aBot.observe(round, ques=doc_tensor, quesLens=doc_lens)
    
    
    answers, ansLens = aBot.forwardDecode(
        beamSize=beamSize, inference='greedy')
    aBot.observe(round, ans=answers, ansLens=ansLens)
    qBot.observe(round, ans=answers, ansLens=ansLens)
    
    # Printing
    print("Q%d: "%(round+1), to_str_gt(questions[0]))
    print("A%d: "%(round+1), to_str_gt(answers[0]))
    summ, summLens = qBot.predictSummary(
            beamSize=beamSize, inference='greedy')
    print('----------------------------------------')
    print("Summary: ", to_str_gt(summ[0]))


summ, summLens = qBot.predictSummary(
            beamSize=beamSize, inference='greedy')
print('----------------------------------------')
print("Summary: ", to_str_gt(summ[0]))

----------------------------------------
Summary:  ( Diagnosis ) : Heart symptoms may several factors may involved , : - Biological differences . People bipolar ( UNK ) . A blood small placenta baby born . A also found UNK UNK UNK . UNK


IndexError: list index out of range

In [14]:
to_str_gt(summ[0])

'( Diagnosis ) : Heart may increase disorder : - several pregnancy cause heart heart type body . UNK ) . UNK cause health . This fungus found UNK UNK UNK UNK - UNK blood pressure UNK . Thi'

In [15]:
Rouge().get_scores(summary, to_str_gt(summ[0]), avg=True)

NameError: name 'Rouge' is not defined

In [18]:
Rouge().get_scores(to_str_gt(summ[0]), docs, avg=True)

{'rouge-1': {'r': 0.024734982332155476,
  'p': 0.30434782608695654,
  'f': 0.045751632596650896},
 'rouge-2': {'r': 0.002188183807439825,
  'p': 0.03125,
  'f': 0.004089978326956174},
 'rouge-l': {'r': 0.024734982332155476,
  'p': 0.30434782608695654,
  'f': 0.045751632596650896}}

In [24]:


for word in summary.split(' '):
    if word in to_str_gt(summ[0]).split(' '):
        print(word)

-
-
-
-
-
heart
-
blood
pressure
-
cause
-


In [16]:
from rouge import Rouge